In [ ]:
import numpy as np
import os

from scipy.spatial import distance
import matplotlib.pyplot as plt
import networkx as nx
import copy
from scipy import stats
from copy import deepcopy
import time
import re
#check


In [2]:
def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ]
    return sorted(data, key=alphanum_key)

In [3]:
def extract_position(fname):
    neuron_data = np.loadtxt(fname)
    #print(fname)
    #print(neuron_data.shape)
    if neuron_data.ndim == 1:#  there exists files which do not have any neurites
        return np.zeros((1,3)), neuron_data[2:5]
    #else:
    soma = neuron_data[0,2:5] # considering only the positions/coordinates
    #print(soma)
    neurites = neuron_data[1:,2:5]
    return neurites,soma # neurites refers to either axon terminals/dendrites. It is a loose term.

In [4]:
def count_connections(axonTerminalsA, dendritesB, synaptic_cleft):
    # cdist is used to calculate euclidean distance between axon terminals and neurites
    # dist_axonTerminalsA_dendritesB is 2d array of dimension nrows(axonTerminalsA) x nrows(dendritesB)
    dist_axonTerminalsA_dendritesB = distance.cdist(axonTerminalsA, dendritesB)
    #count all the dist less than 3um as a connection btw axon terminal A and dendrite B
    connections = np.count_nonzero(dist_axonTerminalsA_dendritesB <= synaptic_cleft)
    return connections

In [5]:
def extract_adjacency(file_path_axon, file_path_dendrite, synaptic_cleft):

    file_list_axon=[f for f in sorted_alphanumeric(os.listdir(file_path_axon)) if f.endswith('.swc')
                and os.path.isfile(os.path.join(file_path_axon, f))]
    file_list_dendrite=[f for f in sorted_alphanumeric(os.listdir(file_path_dendrite)) if f.endswith('.swc')
                        and os.path.isfile(os.path.join(file_path_dendrite, f))]# redundant but simply taken

    neuron_names = []
    n_neurons = len(file_list_axon)
    adjacency = np.zeros((n_neurons,n_neurons))

    start = time.time()
    counter = 0


    for i in range(n_neurons):
        j_list = list(range(i+1, n_neurons))

        for j in j_list:

            axonTerminalsA, somaA = extract_position(file_path_axon+file_list_axon[i])
            dendritesB, somaB = extract_position(file_path_dendrite+file_list_dendrite[j])

            axonTerminalsB, somaB = extract_position(file_path_axon+file_list_axon[j])
            dendritesA, somaA = extract_position(file_path_dendrite+file_list_dendrite[i])


            if np.all(axonTerminalsA==0):
                adjacency[i,j] = 0
            else:
                adjacency[i,j] = count_connections(axonTerminalsA, dendritesB, synaptic_cleft)

            if np.all(axonTerminalsB==0):
                adjacency[j,i] = 0
            else:
                adjacency[j,i] = count_connections(axonTerminalsB, dendritesA, synaptic_cleft)

        counter+=1

        if (counter%100)==0:
            end=time.time()
            print(i, "%.3f min"%((end-start)/60))
            start = time.time()

        neuron_names.append(file_list_axon[i][:-4])

    return adjacency, neuron_names

In [6]:
file_path_axon = "./dendrites_axonTermials/axon_terminals/"
file_path_dendrite = "./dendrites_axonTermials/input_dendrites/"
synaptic_cleft=5

adjacency, neuron_names = extract_adjacency(file_path_axon, file_path_dendrite, synaptic_cleft=synaptic_cleft)

FileNotFoundError: [Errno 2] No such file or directory: './dendrites_axonTermials/axon_terminals/'

In [16]:
data = np.load("adjacency_synaptic_cleft_5.000um.npy", allow_pickle=True)
print(data.shape)
print(data.dtype)

(3163, 3163)
int16


In [14]:
data

array([[ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ..., 50,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       ...,
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  7,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0]], dtype=int16)

In [17]:
import numpy as np
import pandas as pd

# Load adjacency data
data = np.load("adjacency_synaptic_cleft_5.000um.npy", allow_pickle=True)

# Load neuron names from a plain text file
with open("neuron_names.txt", "r") as f:
    neuron_names = [line.strip() for line in f.readlines()]  # Read lines and remove extra spaces/newlines

# Verify dimensions
print(f"Data shape: {data.shape}")
print(f"Neuron names count: {len(neuron_names)}")

# Ensure dimensions match
if data.shape[0] != len(neuron_names):
    raise ValueError("The number of neurons does not match the adjacency matrix dimensions.")

# Create a DataFrame
df = pd.DataFrame(data, index=neuron_names, columns=neuron_names)

# Save to Excel
output_file = "adjacency_with_neuron_names.xlsx"
df.to_excel(output_file, index=True)

print(f"Data successfully saved to {output_file}")

Data shape: (3163, 3163)
Neuron names count: 3163
Data successfully saved to adjacency_with_neuron_names.xlsx
